# Automated Reasoning Policy Playground

This notebook demonstrates how to create and manage automated reasoning policies using the AWS Bedrock API. It includes the following functionality:

1. Setting up the Bedrock client with custom API models
2. Creating an automated reasoning policy
3. Starting a build workflow to process a document
4. Monitoring the build workflow status
5. Retrieving the policy definition
6. Extracting rules from the policy document

## Setup

First, let's import the necessary libraries and set up the Bedrock client with custom API models.

In [ ]:

%pip install -r requirements.txt


In [ ]:
import os
import json
import boto3
import uuid
import time
import pandas as pd
from IPython.display import display, HTML, JSON
import ipywidgets as widgets
from datetime import datetime

In [ ]:
REGION_NAME="us-west-2" # Fill in the AWS Region
my_session = boto3.session.Session()
runtime_client = my_session.client('bedrock-runtime', region_name=REGION_NAME)
bedrock_client = my_session.client('bedrock', region_name=REGION_NAME)

## Create a Policy

Now, let's create an automated reasoning policy using the Bedrock API.

In [ ]:
def create_automated_reasoning_policy(policy_name=None, description=None):
    """
    Create an automated reasoning policy using the Bedrock API.
    
    Args:
        policy_name (str, optional): Name of the policy. If not provided, a random name will be generated.
        description (str, optional): Description of the policy.
        
    Returns:
        dict: Response from the API call.
    """
    if policy_name is None:
        policy_name = f"policy{uuid.uuid4().hex[:6]}"
        
    if description is None:
        description = f"Policy created from Jupyter notebook on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    
    print(f"Creating policy: {policy_name}")
    print(f"Description: {description}")
    
    try:
        response = bedrock_client.create_automated_reasoning_policy(
            name=policy_name,
            description=description,
            clientRequestToken=str(uuid.uuid4())
        )
        
        policy_arn = response['policyArn']
        policy_id = policy_arn.split('/')[-1]
        
        print(f"Policy created successfully!")
        print(f"Policy ARN: {policy_arn}")
        print(f"Policy ID: {policy_id}")
        
        return response
    except Exception as e:
        print(f"Error creating policy: {str(e)}")
        raise

In [ ]:
# Create a policy
policy_response = create_automated_reasoning_policy(
    policy_name="test-jupyter-policy-demo",
    description="Policy created from Jupyter notebook demo"
)

# Extract policy ARN and ID
policy_arn = policy_response['policyArn']
policy_id = policy_arn.split('/')[-1]

# Display the response as JSON
JSON(policy_response)

## Upload a Document and Start a Build Workflow

Now, let's upload a document and start a build workflow to process it.

In [ ]:
def read_document(file_path):
    """
    Read a document from a file.
    
    Args:
        file_path (str): Path to the document file.
        
    Returns:
        bytes: Document content.
        str: Document content type.
    """
    try:
        with open(file_path, 'rb') as f:
            document_content = f.read()
        
        # Determine document content type based on file extension
        if file_path.endswith('.pdf'):
            content_type = 'pdf'
        elif file_path.endswith('.txt'):
            content_type = 'txt'
        elif file_path.endswith('.json'):
            content_type = 'txt'  # JSON is treated as text
        else:
            content_type = 'txt'  # Default to text
        
        return document_content, content_type
    except Exception as e:
        print(f"Error reading document: {str(e)}")
        raise

In [ ]:
def start_build_workflow(policy_arn, document_path, instructions):
    """
    Start a build workflow to process a document.
    
    Args:
        policy_arn (str): ARN of the policy.
        document_path (str): Path to the document file.
        instructions (str): Instructions for policy creation.
        
    Returns:
        dict: Response from the API call.
    """
    try:
        # Read the document content
        document_content, content_type = read_document(document_path)
        
        print(f"Starting build workflow for policy: {policy_arn}")
        print(f"Document: {os.path.basename(document_path)} ({content_type})")
        print(f"Instructions: {instructions}")
        
        response = bedrock_client.start_automated_reasoning_policy_build_workflow(
            policyArn=policy_arn,
            buildWorkflowType='INGEST_CONTENT',
            clientRequestToken=str(uuid.uuid4()),
            sourceContent={
                'workflowContent': {
                    'documents': [{
                        'document': document_content,
                        'documentContentType': content_type,
                        'documentName': os.path.basename(document_path),
                        'documentDescription': instructions
                    }]
                }
            }
        )
        
        build_workflow_id = response['buildWorkflowId']
        print(f"Build workflow started successfully!")
        print(f"Build workflow ID: {build_workflow_id}")
        
        return response
    except Exception as e:
        print(f"Error starting build workflow: {str(e)}")
        raise

In [ ]:
# Path to the document file
document_path = 'uploads/sample_medical_policy.pdf'

# Instructions for policy creation
instructions = """
Create a policy that enforces the rules in the provided document.
The policy should focus on data protection, access control, and compliance requirements.
"""

# Start a build workflow
build_workflow_response = start_build_workflow(
    policy_arn=policy_arn,
    document_path=document_path,
    instructions=instructions
)

# Extract build workflow ID
build_workflow_id = build_workflow_response['buildWorkflowId']

# Display the response as JSON
JSON(build_workflow_response)

## Monitor Build Workflow Status

Now, let's monitor the build workflow status using a progress bar and periodic updates.

In [ ]:
def check_build_workflow_status(policy_arn, build_workflow_id):
    """
    Check the status of a build workflow.
    
    Args:
        policy_arn (str): ARN of the policy.
        build_workflow_id (str): ID of the build workflow.
        
    Returns:
        dict: Response from the API call.
    """
    try:
        response = bedrock_client.get_automated_reasoning_policy_build_workflow(
            policyArn=policy_arn,
            buildWorkflowId=build_workflow_id
        )
        
        return response
    except Exception as e:
        print(f"Error checking build workflow status: {str(e)}")
        raise

In [ ]:
def check_policy_status(policy_arn):
    """
    Check the status of a policy.
    
    Args:
        policy_arn (str): ARN of the policy.
        
    Returns:
        dict: Response from the API call.
    """
    try:
        # Get policy details
        policy = bedrock_client.get_automated_reasoning_policy(
            policyArn=policy_arn
        )
        
        # Try to export the policy definition to check if it's active
        try:
            export_response = bedrock_client.export_automated_reasoning_policy_version(
                policyArn=policy_arn
            )
            
            # If we get here, the policy is active
            policy_status = 'ACTIVE'
            policy['status'] = policy_status
            policy['policy_definition'] = export_response.get('policyDefinition', {})
        except Exception as e:
            # Policy is still processing
            policy_status = 'PROCESSING'
            policy['status'] = policy_status
        
        return policy
    except Exception as e:
        print(f"Error checking policy status: {str(e)}")
        raise

In [ ]:
# Create widgets for monitoring
status_output = widgets.Output()
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    orientation='horizontal'
)
status_text = widgets.HTML(value="<b>Status:</b> Initializing...")

# Display the widgets
display(status_text)
display(progress_bar)
display(status_output)

# Monitor the build workflow status
max_attempts = 30
poll_interval = 10  # seconds
policy_active = False

for attempt in range(max_attempts):
    # Check build workflow status
    build_workflow_response = check_build_workflow_status(policy_arn, build_workflow_id)
    build_status = build_workflow_response.get('status', 'UNKNOWN')
    
    # Check policy status
    policy_response = check_policy_status(policy_arn)
    policy_status = policy_response.get('status', 'UNKNOWN')
    
    # Calculate progress
    progress = 0
    if build_status == 'SCHEDULED':
        progress = 10
    elif build_status == 'PREPROCESSING':
        progress = 20
    elif build_status == 'BUILDING':
        progress = 50
    elif build_status == 'TESTING':
        progress = 75
    elif build_status == 'COMPLETED':
        progress = 90
    
    if policy_status == 'ACTIVE':
        progress = 100
        policy_active = True
    
    # Update the widgets
    progress_bar.value = progress
    status_text.value = f"<b>Status:</b> Build workflow: {build_status}, Policy: {policy_status}"
    
    with status_output:
        print(f"Check {attempt + 1}: Build workflow status: {build_status}, Policy status: {policy_status}")
    
    # If the policy is active, we're done
    if policy_active:
        progress_bar.bar_style = 'success'
        break
    
    # If the build workflow failed, we're done
    if build_status in ['FAILED', 'CANCELLED']:
        progress_bar.bar_style = 'danger'
        with status_output:
            print(f"Build workflow failed or was cancelled.")
        break
    
    # Wait before the next check
    if attempt < max_attempts - 1:
        time.sleep(poll_interval)

# Final status update
if policy_active:
    status_text.value = f"<b>Status:</b> Policy is ACTIVE and ready to use!"
else:
    status_text.value = f"<b>Status:</b> Policy is not yet active. Build workflow status: {build_status}"

## Retrieve Policy Definition

Now, let's retrieve the policy definition once the policy is active.

In [ ]:
def get_policy_definition(policy_arn):
    """
    Get the policy definition.
    
    Args:
        policy_arn (str): ARN of the policy.
        
    Returns:
        dict: Policy definition.
    """
    try:
        response = bedrock_client.export_automated_reasoning_policy_version(
            policyArn=policy_arn
        )
        
        return response.get('policyDefinition', {})
    except Exception as e:
        print(f"Error getting policy definition: {str(e)}")
        raise

In [ ]:
## You need to save the new policy as a version to be able to get the policy definition
import pprint
# Get the policy definition
if policy_active:
    policy_definition = get_policy_definition(policy_arn)
    print(f"Policy definition retrieved successfully!")
    
    # Display the policy definition as JSON
    pprint.pprint(policy_definition, compact=True)
else:
    print("Policy is not yet active. Cannot retrieve policy definition.")

In [ ]:
def extract_rules_from_document(document_path):
    """
    Extract rules from the policy document
    
    Args:
        document_path (str): Path to the document file.
        
    Returns:
        list: Extracted rules.
    """
    document_format = document_path.split('.')[-1].lower()
    rules = []
    
    try:
        # For JSON format
        if document_format == 'json':
            with open(document_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
                # Extract rules from the JSON structure
                if 'policy_sections' in data:
                    for section in data['policy_sections']:
                        section_id = section.get('section_id', '')
                        section_title = section.get('title', '')
                        
                        if 'rules' in section:
                            for rule in section['rules']:
                                rule_id = rule.get('rule_id', '')
                                description = rule.get('description', '')
                                examples = rule.get('examples', [])
                                
                                rules.append({
                                    'section_id': section_id,
                                    'section_title': section_title,
                                    'rule_id': rule_id,
                                    'description': description,
                                    'examples': examples
                                })
        
        # For text format
        elif document_format == 'txt':
            with open(document_path, 'r', encoding='utf-8') as f:
                content = f.read()
                
                # Parse rules from text format
                # This is a simple parser that looks for "Rule X.Y:" patterns
                import re
                
                # Find sections
                section_pattern = r'SECTION (\d+): ([^\n]+)'
                sections = re.findall(section_pattern, content)
                
                # Find rules
                rule_pattern = r'Rule (\d+\.\d+): ([^\n]+)'
                rule_matches = re.findall(rule_pattern, content)
                
                # Map rules to sections
                for rule_id, description in rule_matches:
                    section_id = rule_id.split('.')[0]
                    section_title = next((title for sid, title in sections if sid == section_id), '')
                    
                    # Find examples for this rule
                    example_pattern = f"Rule {rule_id}:.*?Examples:(.*?)(?=Rule|$)"
                    example_match = re.search(example_pattern, content, re.DOTALL)
                    examples = []
                    
                    if example_match:
                        examples_text = example_match.group(1).strip()
                        examples = [ex.strip() for ex in examples_text.split(',')]
                    
                    rules.append({
                        'section_id': section_id,
                        'section_title': section_title,
                        'rule_id': rule_id,
                        'description': description,
                        'examples': examples
                    })
        
        # For PDF format (not implemented in this demo)
        elif document_format == 'pdf':
            # In a real application, you would use a PDF parsing library
            # For this demo, we'll return an empty list
            print("PDF parsing is not implemented")
    
    except Exception as e:
        print(f"Error extracting rules: {str(e)}")
    
    return rules

In [ ]:
# Extract rules from the document
rules = extract_rules_from_document(document_path)
print(f"Extracted {len(rules)} rules from the document.")

# Display the rules as a DataFrame
if rules:
    rules_df = pd.DataFrame(rules)
    display(rules_df)
else:
    print("No rules extracted from the document.")

## Conclusion

In this notebook, we've demonstrated how to:

1. Set up the Bedrock client with custom API models
2. Create an automated reasoning policy
3. Start a build workflow to process a document
4. Monitor the build workflow status
5. Retrieve the policy definition
6. Extract rules from the policy document

This provides a foundation for working with automated reasoning policies in AWS Bedrock.